# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [ ]:
! pip install -q schedule pytest # установка библиотек, если ещё не

In [4]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import requests
import schedule
import sys
import pytest
from bs4 import BeautifulSoup
from urllib.parse import urljoin

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [5]:
def get_book_data(url: str) -> dict:
    """
    Функция парсит HTML-страницу и извлекает информацию по книге:
        - название;
        - описание;
        - цена;
        - рейтинг;
        - количество в наличии;
        - характеристики из таблицы Product Information.
   
    Аргумент: url (str): URL web-страницы.
    
    Возвращает словарь с ключами:
            - "title" (str): Название книги.
            - "description" (str | None): Описание книги (если присутствует).
            - "price" (str): Цена книги.
            - "availability" (str): Наличие и количество на складе.
            - "rating" (str): Рейтинг книги.
            - "product_information" (dict): Характеристики из таблицы Product Information.
    """
    response = requests.get(url)
    response.raise_for_status()
    
    response.encoding = 'utf-8' # Декодирование в UTF-8
    
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find('h1').get_text(strip=True)    # Название книги

    # Описание книги
    description = None
    desc_header = soup.find('div', id='product_description')
    if desc_header and desc_header.find_next_sibling('p'):
        description = desc_header.find_next_sibling('p').get_text(strip=True)
    
    price = soup.find('p', class_='price_color').get_text(strip=True)   # Цена книги
    availability = soup.find('p', class_='instock availability').get_text(strip=True)   # Наличие книги

    # Рейтинг книги
    rating_element = soup.find('p', class_='star-rating')
    rating = rating_element['class'][1] if rating_element and len(rating_element['class']) > 1 else None

    # Дополнительные характеристики из таблицы Product Information
    product_info = {}
    table = soup.find('table', class_='table table-striped')
    if table:
        for row in table.find_all('tr'):
            key = row.find('th').get_text(strip=True)
            value = row.find('td').get_text(strip=True)
            product_info[key] = value

    return {
        'title': title,
        'price': price,
        'availability': availability,
        'rating': rating,
        'description': description,
        'product_information': product_info,
          }

def print_book_data(data: dict) -> None:
    """
    Функция оформляет вывод данных о книге.

    Аргумент: data (dict): Словарь, возвращённый функцией get_book_data().
    """
    sys.stdout.reconfigure(encoding='utf-8') # Вывод в UTF-8
    
    print("\n" + "-" * 50)
    print(f"  {data['title']}")
    print("-" * 50)
    print(f" Цена: {data['price']}")
    print(f" Рейтинг: {data['rating']}")
    print(f" Наличие: {data['availability']}\n")

    if data['description']:
        print(" Описание:")
        print(f"{data['description']}\n")

    print(" Характеристики (Product Information):")
    for key, value in data['product_information'].items():
        print(f"  • {key}: {value}")
    print("-" * 50 + "\n")

In [6]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'title': 'A Light in the Attic',
 'price': '£51.77',
 'availability': 'In stock (22 available)',
 'rating': 'Three',
 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe 

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [13]:
def scrape_books(url: str, is_save: bool = False) -> list:
    """
    Функция проходиться по всем страницам из каталога и собирает данные о книгах.

    Аргументы:
        url (str): Базовый URL каталога, где {N} номер страницы.
        is_save (bool): Если True, сохраняет данные в файл 'books_data.txt'.
                        Если False, просто возвращает результат.

    Возвращает:
        list: Список словарей с данными о книгах.
    """
 # НАЧАЛО ВАШЕГО РЕШЕНИЯ   
    all_books = []
    page_num = 1

    print("Начинаем парсинг каталога: \n")

    while True:
        page_url = url.format(page_num)
        response = requests.get(page_url)
        base_page_url = page_url.rsplit('/', 1)[0] + '/'

        if response.status_code == 404:
            print(f" Парсинг завершён. Всего страниц: {page_num - 1}")
            break

        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')

        book_links = [
            urljoin(base_page_url, a['href'])
            for a in soup.select('h3 a')
        ]

        print(f" Обрабатываем страницу {page_num} ({len(book_links)} книг)...")

        for link in book_links:
            try:
                book_data = get_book_data(link)
                all_books.append(book_data)
            except Exception as e:
                print(f" Ошибка при обработке {link}: {e}")

        page_num += 1

    if is_save:
        with open("books_data.txt", "w", encoding="utf-8") as file:
            for i, book in enumerate(all_books, start=1):
                file.write(f"{i}. {book['title']} — {book['price']} — {book['availability']}\n")
                file.write(f"   Рейтинг: {book['rating']}\n")
                if book['description']:
                    file.write(f"   Описание: {book['description'][:200]}...\n")
                file.write(f"   Характеристики: {book['product_information']}\n")
                file.write("-" * 80 + "\n")
        print("\n Данные успешно сохранены в файл books_data.txt")

    return all_books
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [14]:
# Проверка работоспособности функции
res = scrape_books("https://books.toscrape.com/catalogue/page-{}.html", is_save=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

Начинаем парсинг каталога: 

 Обрабатываем страницу 1 (20 книг)...
 Обрабатываем страницу 2 (20 книг)...
 Обрабатываем страницу 3 (20 книг)...
 Обрабатываем страницу 4 (20 книг)...
 Обрабатываем страницу 5 (20 книг)...
 Обрабатываем страницу 6 (20 книг)...
 Обрабатываем страницу 7 (20 книг)...
 Обрабатываем страницу 8 (20 книг)...
 Обрабатываем страницу 9 (20 книг)...
 Обрабатываем страницу 10 (20 книг)...
 Обрабатываем страницу 11 (20 книг)...
 Обрабатываем страницу 12 (20 книг)...
 Обрабатываем страницу 13 (20 книг)...
 Обрабатываем страницу 14 (20 книг)...
 Обрабатываем страницу 15 (20 книг)...
 Обрабатываем страницу 16 (20 книг)...
 Обрабатываем страницу 17 (20 книг)...
 Обрабатываем страницу 18 (20 книг)...
 Обрабатываем страницу 19 (20 книг)...
 Обрабатываем страницу 20 (20 книг)...
 Обрабатываем страницу 21 (20 книг)...
 Обрабатываем страницу 22 (20 книг)...
 Обрабатываем страницу 23 (20 книг)...
 Обрабатываем страницу 24 (20 книг)...
 Обрабатываем страницу 25 (20 книг)...
 Обра

## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [9]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
def scheduler (url: str, time_str: str = "19:00"):
    """
    Настраивает ежедневный запуск парсера в указанное время и выполняет сбор данных с указанного URL.

    Аргументы:
        url (str): Базовый URL каталога.
        time_str (str): Время запуска в 19:00.
    """
    def time():
        scrape_books(url, is_save=True) 

    schedule.every().day.at(time_str).do(time)
    print(f" Планировщик запущен. \n")

    while True:
        schedule.run_pending()
        time.sleep(60)  # проверяем расписание раз в минуту
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [10]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
!pytest tests/test_scraper.py

============================= test session starts =============================
platform win32 -- Python 3.13.5, pytest-8.4.2, pluggy-1.6.0
rootdir: C:\Windows\System32\1
plugins: anyio-4.11.0
collected 3 items

tests\test_scraper.py ...                                                [100%]

============================== 3 passed in 2.68s ==============================


## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```